In [61]:
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt

In [62]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472


[WDM] - Driver [C:\Users\annar\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [63]:
def create_soup(url):
    browser.visit(url)
    # Create BeautifulSoup object; parse with 'html.parser'
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    return soup

# NASA Mars News
* Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [4]:
url = 'https://redplanetscience.com/'
soup = create_soup(url)

In [5]:
title = soup.find('div' , class_="content_title").text

news_p = soup.find('div', class_="article_teaser_body").text

news_dict = {'title':title,
                'news_p':news_p}


news_dict

{'title': "NASA's Mars Perseverance Rover Gets Its Sample Handling System",
 'news_p': 'The system will be collecting and storing Martian rock and soil. Its installation marks another milestone in the march toward the July launch period.'}

# JPL Mars Space Images - Featured Image

* Visit the url for the Featured Space Image site here.

* Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a     variable called featured_image_url.

* Make sure to find the image url to the full size .jpg image.

* Make sure to save a complete url string for this image.

In [34]:
space_url = 'https://spaceimages-mars.com/'
soup = create_soup(space_url)

In [35]:
try:
    target = 'button[class="btn btn-outline-light"]'
    browser.find_by_tag(target).click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image_src = soup.find('img', class_="fancybox-image")['src']

except:
    print('can\'t find the image')    
featured_image_url = space_url + image_src

featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

# Mars Facts

* Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter,     Mass, etc.

* Use Pandas to convert the data to a HTML table string.

In [9]:
mars_url = 'https://galaxyfacts-mars.com/'
# using pandas read html
tables = pd.read_html(mars_url)
mars_fact_df = tables[0]

In [10]:
mars_fact_df.columns = ['Description','Mars', 'Earth']
mars_fact_df = mars_fact_df.iloc[1:]
mars_fact_df.set_index('Description', drop=True , inplace= True)

mars_fact_df.head()

,Mars,Earth
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days


# Mars Hemispheres

* Visit the astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

* You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

* Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere     name. Use a Python dictionary to store the data using the keys img_url and title.

* Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for   each hemisphere.

In [64]:
# set url
hem_url = 'https://marshemispheres.com/'
soup = create_soup(hem_url)
# get soup


items = soup.find_all('div', class_="item")

hemisphere_urls = []

for item in items:
    hmsphere = {}
    name = item.h3.text
#     link = item.a['href']

    # get full image
    try:
        browser.links.find_by_partial_text(name).click()
        print(browser.url)
        html2 = browser.html
        imgsoup = BeautifulSoup(html2, 'html.parser')
        imgsoup
        img = imgsoup.find('img', class_="wide-image")
        
        hmsphere['title'] = name[:-9]
        hmsphere['img_url'] = hem_url + img['src']
          
    except:
        print("Could not get Image Link")   
        
    hemisphere_urls.append(hmsphere)    
    browser.back()

browser.quit()

print(hemisphere_urls)

https://marshemispheres.com/cerberus.html
https://marshemispheres.com/schiaparelli.html
https://marshemispheres.com/syrtis.html
https://marshemispheres.com/valles.html
[{'title': 'Cerberus Hemisphere', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
